## CSV (Comma Separated Values)

CSV ist ein weit verbreitetes Format zur Speicherung tabellarischer Daten (also Zeilen und Spalten). Jede Zeile repräsentiert dabei einen Datensatz,
jede Spalte einen Wert dieses Datensatzen. Die einzelnen Werte sind dabei durch ein Trennzeichen (default: Komma) voneinander getrennt:

```
Hans,Huber,Graz,2001-09-06
Anna Maria,Bauer,Weiz,1999-02-14
```

Auf den ersten Blick erscheint es ziemlich einfach, so eine Datei einzulesen und beispielsweise als Liste von Listen zu verwalten:

In [ ]:
data = []
with open('data/beispiel1.csv', encoding="utf-8") as fh:
    for line in fh:
        data.append(line.rstrip().split(','))
print(data) 

In der Praxis ist es aber oft nicht ganz so einfach, etwa wenn das Trennzeichen in einem Wert vorkommt:

```
Hans,Huber,"Graz, Wien",2001-09-06
Anna Maria,Bauer,Weiz,1999-02-14
```

Daher soll man nicht versuchen, selbst Code zu schreiben, der ein solches Format parst, sondern auf bestehende Bibliotheken zurückgreifen. In der Standardlibrary gibt es dafür das `csv` Modul. Ebenfalls sehr populär ist ist die `read_csv()` Funktion der Pandas Bibliothek, deren Verwendung aber nur Sinn macht, wenn man ohnehin mit Pandas arbeiten will. Daher beschäftigen wir uns hier mit dem csv Modul. Die Dokumentation des Moduls finden Sie hier: https://docs.python.org/3/library/csv.html.

### CSV lesen

Zum Einlesen einer CSV-Datei benötigen wir einen Reader, dem wir die CSV-Datei als Parameter übergeben:

In [ ]:
import csv

with open('data/cities.csv', encoding="utf-8") as fh:
    reader = csv.reader(fh)

Das `reader` Objekt stellt nun die einzelnen Zeilenwerte als Folge von Listen bereit:

In [ ]:
import csv

with open('data/cities.csv', encoding='utf-8') as fh:
    reader = csv.reader(fh)
    for row in reader:
        print(row)

Damit kann man gezielt auf Spaltenwerte zugreifen. Wenn ich etwas nur an den Städtenamen und Bundesländern interessiert bin:

In [ ]:
import csv

with open('data/cities.csv', encoding='utf-8') as fh:
    reader = csv.reader(fh)
    next(reader) # skip the first line
    for row in reader:
        print(f"{row[0]}, {row[2]}, {row[-1]}")

Es ist zu beachten, dass alle Werte den Datentyp `str` haben. Im nächsten Beispiel speichern wir alle Zeilen in einer Liste `data`. Dann versuchen wir die Summe der aktuellen Einwohnerzahlen zu bilden, was nicht funktioniert, weil `sum()` numerische Werte (und keine Strings) erwartet:

In [ ]:

with open('data/cities.csv', encoding='utf-8') as fh:
    reader = csv.reader(fh)
    next(reader) # skip the first line
    data = list(reader)

sum_of_population = sum([row[-1] for row in data])        

Damit dieser Code funktioniert, müssen wir explizit eine Typumwandlung durchführen (und den Tausender-Trenner entfernen):

In [ ]:
sum_of_population = sum([int(row[-1].replace('.', '')) for row in data])
print(sum_of_population)

Eine alternative Strategie wäre, die Daten schon zu Beginn in eine brauchbare Form zu bringen. 
Das empfiehlt sich vor allem, wenn weitere Auswertungen auf diesen Daten benötigt werden.

In [ ]:
data = []
with open('data/cities.csv', encoding='utf-8') as fh:
    reader = csv.reader(fh)
    next(reader) # skip the first line
    for row in reader:
        data.append((row[0], row[1], row[2], int(row[3]), 
                    int(row[4].replace('.', '')), 
                    int(row[5].replace('.', '')),
                    int(row[6].replace('.', ''))))

sum_of_population = sum([row[-1] for row in data])
print(sum_of_population)

#### Reader konfigurieren

Wir haben das `reader` Objekt bisher in seiner einfachsten Form verwendet. Es bietet aber eine Reihe von Möglichkeiten, über die das Parsen der Input-Daten beeinflusst werden können. Beispielsweise kann man über den Parameter `delimiter`= 
das Trennzeichen ändern. Die vollständige Liste an Möglichkeiten finden Sie hier: https://docs.python.org/3/library/csv.html?highlight=csv#csv-fmt-params.

#### CSV als Liste von Dictionaries

Normalerweise stellt das csv Modul die Daten als Liste von Listen bereit. Manchmal ist es aber übersichtlicher, auf einer Liste von Dictionaries zu operieren, in der die Keys die Spaltennamen sind. Das csv Modul stellt das einen speziellen `DictReader` zur Verfügung:

In [ ]:
with open('data/cities.csv', encoding="utf-8") as fh:
    reader = csv.DictReader(fh)
    for row in reader:
        print(row['name'], row['province'], row['population_2023'])

Der `DictReader` verwendet automatisch die Felder der ersten Zeile als keys. Falls die CSV-Datei keine Feldnamen in der ersten Zeile hat, können diese explizit als Liste von Strings angegeben werden:

In [ ]:
with open('data/cities_no_fieldnames.csv', encoding="utf-8") as fh:
    reader = csv.DictReader(fh, fieldnames=["Name", "Bezirk", "Bundesland", "Stadt seit", 
                                            "Einwohner 2001", "Einwohner_2011", "Einwohner 2023"])
    for row in reader:
        print(row['Name'], row['Bundesland'], row['Einwohner 2023'])

## CSV schreiben

Auch beim Schreiben von CSV empfehle ich dringend, auf fertige Bibliotheken zurückzugreifen, weil diese mit Besonderheiten wie Trennzeichen in den Daten gut umgehen können. Das csv Modul bietet einige Writer an.

Nehmen wir an, wir hätten in unserem Programm Daten generiert oder aus anderen Daten abgeleitet. Da diese in Tabellenform vorliegen, wollen wir sie in eine CSV Datei speichern:

In [ ]:
data = [
    ("foo", 123, "a, c, x"),
    ("bar", 987, "a, r, z"),
    ("foobar", 1245, "b, m")
]
with open('output/data.csv', 'w', encoding="utf-8")  as fh:
    writer = csv.writer(fh)
    writer.writerows(data)

`writer.writerows()` schreibt alles Elemente der Liste in unsere zum Schreiben geöffnete CSV-Datei `output/cities.csv`. Überprüfen Sie, ob und wie die Daten korrekt geschrieben wurden:

In [ ]:
with open('output/data.csv') as fh:
   print(fh.read())

Falls wir nur einzelne Zeilen schreiben wollen, sollten wir `writer.writerow()` verwenden:

In [ ]:
data = [
    ("foo", 123, "a, c, x"),
    ("bar", 987, "a, r, z"),
    ("foobar", 1245, "b, m")
]
with open('output/data.csv', 'w', encoding="utf-8")  as fh:
    writer = csv.writer(fh)
    for row in data:
        if row[1] > 500:
            writer.writerow(row)

In [ ]:
with open('output/data.csv') as fh:
   print(fh.read())

## Vertiefende Literatur

Ich empfehle ausdrücklich, mindestens eine der folgenden Ressourcen zur Vertiefung zu lesen!

  * https://docs.python.org/3/library/csv.html

## Lizenz

This notebook ist part of the course [Grundlagen der Programmierung](https://github.com/gvasold/gdp) held by [Gunter Vasold](https://online.uni-graz.at/kfu_online/wbForschungsportal.cbShowPortal?pPersonNr=51488) at Graz University 2017&thinsp;ff. 

<p>
    It is licensed under <a href="https://creativecommons.org/licenses/by-nc-sa/4.0">CC BY-NC-SA 4.0</a>
</p>

<table>
    <tr>
    <td>
        <img style="height:22px" 
             src="https://mirrors.creativecommons.org/presskit/icons/cc.svg?ref=chooser-v1"/></li>
    </td>
    <td>
    <img style="height:22px;"
         src="https://mirrors.creativecommons.org/presskit/icons/by.svg?ref=chooser-v1" /></li>
    </td>
    <td>
        <img style="height:22px;"
         src="https://mirrors.creativecommons.org/presskit/icons/nc.svg?ref=chooser-v1" /></li>
    </td>
    <td>
        <img style="height:22px;"
             src="https://mirrors.creativecommons.org/presskit/icons/sa.svg?ref=chooser-v1" /></li>
    </td>
</tr>
</table>